In [1]:
import os
import pandas as pd
import numpy as np
import joblib

In [2]:
#!python -m pip install -U pip --user

In [3]:
DATA_PATH = os.path.abspath('./results_1_day_CA_TX_WI/')

# WitWidget

In [4]:
#!pip3 install witwidget
!jupyter nbextension install --py --symlink --sys-prefix witwidget
!jupyter nbextension enable --py --sys-prefix witwidget

Installing C:\Users\priyanshusinha\anaconda3\lib\site-packages\witwidget\static -> wit-widget
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable witwidget --py --sys-prefix
    
Enabling notebook extension wit-widget/extension...
      - Validating: ok


In [5]:
CSV_PATH = os.path.join(DATA_PATH, 'combined_test_data_points.csv')
MODEL_PATH = os.path.join(DATA_PATH, 'M5_LSTM_CA_TX_WI.h5')
SCALER_PATH = os.path.join(DATA_PATH, 'scaler_instance_mn_mx.bz')

In [6]:
test_x = pd.read_csv(CSV_PATH)

In [7]:
test_x.head()

,CA,TX,WI,snap_CA,snap_TX,snap_WI,sales_price_min,sales_price_max,sales_price_mean,weekday_Friday,...,WI_lagged_24,CA_lagged_25,TX_lagged_25,WI_lagged_25,CA_lagged_26,TX_lagged_26,WI_lagged_26,CA_lagged_27,TX_lagged_27,WI_lagged_27
0,0.03027,0.1367,0.1797,1,1,0,0.8335,0.0,1.000,1,...,0.7456,0.8125,0.7686,0.7456,0.8125,0.7686,0.7456,0.7540,0.6875,0.3647
1,0.03027,0.1367,0.1797,1,1,0,0.8335,0.0,0.625,1,...,0.7456,0.8125,0.7686,0.7456,0.7540,0.6875,0.3647,0.7540,0.6875,0.3647
2,0.03027,0.1367,0.1797,1,1,0,0.8335,0.0,0.000,1,...,0.7456,0.7540,0.6875,0.3647,0.7540,0.6875,0.3647,0.7540,0.6875,0.3647
3,0.49600,0.3682,0.6100,1,0,1,0.8335,0.0,0.000,0,...,0.3647,0.7540,0.6875,0.3647,0.7540,0.6875,0.3647,0.2627,0.3643,0.2524
4,0.49600,0.3682,0.6100,1,0,1,0.8335,0.0,1.000,0,...,0.3647,0.7540,0.6875,0.3647,0.2627,0.3643,0.2524,0.2627,0.3643,0.2524


In [8]:
import tensorflow as tf

In [9]:
from tensorflow.keras.models import load_model

In [10]:
model = load_model(MODEL_PATH)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 256)           370688    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 repeat_vector (RepeatVector  (None, 1, 128)           0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 1, 128)            131584    
                                                                 
 lstm_3 (LSTM)               (None, 1, 256)            394240    
                                                                 
 time_distributed (TimeDistr  (None, 1, 256)           65792     
 ibuted)                                                

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = joblib.load(SCALER_PATH)

In [29]:
def get_prediction(example):
    #print(example)
    start_test = test_x[:28].values
    n_past = 28
    n_future = 1
    test_X = []
    example = np.array(example)
    print(example.shape)
    example_processed = np.concatenate([start_test, example])
    print(example_processed.shape)
    for i in range(n_past, len(example_processed) - n_future + 1):
        test_X.append(example_processed[i - n_past:i, 0:example_processed.shape[1]])
    test_X = np.array(test_X)
    print(test_X.shape)
    prediction = model.predict(test_X)
    return prediction[:, 0]

In [30]:
get_prediction(test_x[27:29].values[:].tolist())

(2, 105)
(30, 105)
(2, 28, 105)
1/1 [==============================] - 0s 21ms/step


array([[4832.366 , 3177.9058, 3024.8374],
       [4809.972 , 3163.1812, 3010.8225]], dtype=float32)

In [31]:
import witwidget

In [32]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [36]:
config_builder = WitConfigBuilder(test_x[27:].values[:].tolist(), feature_names = test_x.columns.tolist()).set_custom_predict_fn(get_prediction)

In [37]:
ww = WitWidget(config_builder, height=800)

In [38]:
ww.render()

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'feature_names': ['CA', 'TX', 'WI', 'snap…

(290, 105)
(318, 105)
(290, 28, 105)
10/10 [==============================] - 0s 21ms/step
(1, 105)
(29, 105)
(1, 28, 105)
1/1 [==============================] - 0s 22ms/step
